In [12]:
import inspect
from functools import lru_cache
from collections import OrderedDict
from inspect import Parameter
from functools import wraps
from typing import *

from mousse import validate, parse

In [58]:
@lru_cache
def get_func_validator(func: Callable):
    signature = inspect.signature(func)
    parameters = signature.parameters
    parameters = OrderedDict(parameters.items())
    
    has_var_args = False
    for param in parameters.values():
        if param.kind == param.VAR_POSITIONAL:
            has_var_args = True
            break
            
    if has_var_args:
        parameters.pop(param.name)
    
    has_var_kwargs = False
    for param in parameters.values():
        if param.kind == param.VAR_KEYWORD:
            has_var_kwargs = True
            break
            
    if has_var_kwargs:
        parameters.pop(param.name)

    positional_only_params = [
        param for param in parameters.values()
        if param.kind == param.POSITIONAL_ONLY
    ]
    
    positional_or_keyword_params = [
        param for param in parameters.values()
        if param.kind == param.POSITIONAL_OR_KEYWORD
    ]
    
    def validator(*args, **kwargs) -> Union[bool, str]:
        l_parameters = parameters.copy()
        
        if len(positional_only_params) > len(args):
            return False, "Missing positional only params"
        
        for arg, param in zip(args, positional_only_params):
            l_parameters.pop(param.name)
        
            if param.annotation in (inspect._empty, Any):
                continue

            if not validate(param.annotation, arg):
                return False, f"Wrong type for {param.name}. Expect type {param.annotation.__name__}"
            
        args = args[len(positional_only_params):]
        
        if len(args) > len(positional_or_keyword_params):
            if not has_var_args:
                return False, "Too many arguments"
        
        for arg, param in zip(args, positional_or_keyword_params):
            l_parameters.pop(param.name)
            if param.annotation in (inspect._empty, Any):
                continue
            
            if param.default != param.empty and arg == param.default:
                continue

            if not validate(param.annotation, arg):
                return False, f"Wrong type for {param.name}. Expect type {param.annotation.__name__}"

            
        for key in kwargs:
            if key not in l_parameters:
                if not has_var_kwargs:
                    return False, f"Unknown keyword argument: {key}"
        
        for key, val in kwargs.items():
            if key in l_parameters:
                param = l_parameters.pop(key)
                if param.annotation in (inspect._empty, Any):
                    continue
            
                if param.default != param.empty and val == param.default:
                    continue

                if not validate(param.annotation, val):
                    return False, f"Wrong type for {param.name}. Expect type {param.annotation.__name__}"
            
        for param in l_parameters.values():
            if param.default == inspect._empty:
                return False, f"Missing value for {param.name}"
        
        return True, ""
    
    return validator

In [59]:
def validate_parameters(func: Callable, /, *args, **kwargs) -> Union[bool, str]:
    validator = get_func_validator(func)
    return validator(*args, **kwargs)

In [60]:
def foo(name: str, age: int, school: str, address: str = None, **kwargs):
    print(name, age, school, address)
    
@wraps(foo)
def bar(*args, **kwargs):
    return foo(*args, **kwargs)

In [61]:
def type_checking(func: Callable = None, param_annotation: bool = True, return_annotation: bool = True):    
    def decorator(func: Callable):        
        signature = inspect.signature(func)

        @wraps(func)
        def wrapper(*args, **kwargs):
            if param_annotation:
                valid, err = validate_parameters(func, *args, **kwargs)
                assert valid, err

            output = func(*args, **kwargs)
            if return_annotation and signature.return_annotation is not inspect._empty:
                assert validate(signature.return_annotation, output), \
                    f"Incorrect return type: {type(output)}. Correct return type: {signature.return_annotation}"
        
            return output
        
        return wrapper
    
    if func is not None:
        return decorator(func)
    
    return decorator

In [65]:
@type_checking
def foo(name: str, age: int, school: str, address: str = None, **kwargs):
    print(name, age, school, address)

In [70]:
foo("a", 1, "HUST", None, 1)

AssertionError: Too many arguments

In [39]:
inspect.signature(foo).parameters["address"].kind

<_ParameterKind.POSITIONAL_OR_KEYWORD: 1>

In [6]:
None == None

True

In [76]:
a = (1, 2, 3, 4, 5)
a = iter(a)

In [9]:
a = 1
a is 1

<>:2: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:2: SyntaxWarning: "is" with a literal. Did you mean "=="?
/tmp/ipykernel_12481/3780910533.py:2: SyntaxWarning: "is" with a literal. Did you mean "=="?
  a is 1


True

In [3]:
class Foo:
    def __init__(self, **kwargs):
        for key, val in kwargs.items():
            setattr(self, key, val)
            
    def show(self):
        print(self.a)

In [4]:
a = Foo(a=1, b=2)

In [5]:
a.show()

1
